In [1]:
from typing import List, Dict
from pydantic import BaseModel, Field
from langchain_openai import ChatOpenAI
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import JsonOutputParser

from app.utils.config import settings

/Users/kirtanmataliya/Documents/projects/Adaptive-Learning/app/config/config.ini


In [2]:
class QuizOutput(BaseModel):
    quiz: List[Dict] = Field(description="List of dictionaries containing 5 keys, 'questions', 'options', 'correct option', 'sub_topic' and 'difficulty_level'.")

In [3]:
parser = JsonOutputParser(pydantic_object=QuizOutput)

In [4]:
PROMPT = """You are a quiz generator. Your job is to create {no_questions} quiz question based on the provided topic, subtopics, and the user's proficiency level.

Inputs:
- Topic: {topic} (The main theme of the quiz)
- SubTopics: {subtopics} (Relevant subtopics)

Instructions:
1. Ensure all the generated questions are form the subtopics only.
2. Make sure you cover all the subtopics in the quiz.
3. Quiz should be divided into 3 levels of difficulty: Easy, Medium, and Hard.

Output Format (JSON):
{{
  "quiz": [
    {{
      "question": <question_text>,
      "options": ["A. <option1>", "B. <option2>", "C. <option3>", "D. <option4>"],
      "correct_option": <one_of_A_B_C_or_D>,
      "sub_topic": <subtopic>,
      "difficulty_level": <difficulty_level>
    }},
    ...
  ],
}}

- Provide only the answer in the specified format with no extra explanations.
"""

In [5]:
llm = ChatOpenAI(
    model=settings.OPENAI.model,
    base_url=settings.OPENAI.base_url,
    api_key=settings.OPENAI.api_key
)

In [6]:
prompt = PromptTemplate(
    template=PROMPT,
    input_variables=["no_questions", "topic", "subtopics"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

In [7]:
chain = prompt | llm | parser

In [9]:
ans = chain.invoke(
    {
    "no_questions": 20,
    "topic": "Data Structures and Algorithms",
    "subtopics": ["Arrays", "Linked Lists", "Sorting Algorithms"]
    }
)

In [10]:
ans

{'quiz': [{'question': 'What is the primary purpose of an array in data structures?',
   'options': ['A. To store data in a structured format',
    'B. To manage memory efficiently',
    'C. To implement recursion',
    'D. To handle dynamic data'],
   'correct_option': 'A',
   'sub_topic': 'Arrays',
   'difficulty_level': 'Easy'},
  {'question': 'Which of the following is a characteristic of a singly linked list?',
   'options': ['A. It can be traversed in both directions',
    'B. It uses more memory than a doubly linked list',
    'C. Each node contains only one pointer',
    'D. It is immune to node deletion issues'],
   'correct_option': 'C',
   'sub_topic': 'Linked Lists',
   'difficulty_level': 'Easy'},
  {'question': 'Which sorting algorithm is known for its stability and efficiency?',
   'options': ['A. Quick Sort',
    'B. Merge Sort',
    'C. Heap Sort',
    'D. Selection Sort'],
   'correct_option': 'B',
   'sub_topic': 'Sorting Algorithms',
   'difficulty_level': 'Medium'}